In [2]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import time


In [3]:
text_file_dir = "./data/wikispeedia_articles_plaintext/plaintext_articles/"
file_data = []

for filename in os.listdir(text_file_dir):
    if filename.endswith(".txt"):  # Ensure you're only reading .txt files
        with open(os.path.join(text_file_dir, filename), 'r', encoding='utf-8') as file:
            content = file.read()
        
            # Split the content into lines to remove the header
            lines = content.split('\n')

            # Removing the header (line 0) and join back the content
            if lines:
                title = lines.pop(2)  # Assuming the first line is the title
            
            content = ' '.join(lines).replace("\n", " ")
            
            # Create a dictionary with title and content
            file_data.append({"title": title, "text_content": content})

# Create a DataFrame
project_data = pd.DataFrame(file_data)

# Display the DataFrame structure
print(project_data.head())

article_titles = project_data['title'].tolist()
article_titles


                         title  \
0               Second Crusade   
1               Navassa Island   
2             Evan Rachel Wood   
3  Tropical Storm Henri (2003)   
4      Final Fantasy Adventure   

                                        text_content  
0     #copyright   2007 Schools Wikipedia Selecti...  
1     #copyright   2007 Schools Wikipedia Selecti...  
2     #copyright   2007 Schools Wikipedia Selecti...  
3     #copyright   2007 Schools Wikipedia Selecti...  
4     #copyright   2007 Schools Wikipedia Selecti...  


['Second Crusade',
 'Navassa Island',
 'Evan Rachel Wood',
 'Tropical Storm Henri (2003)',
 'Final Fantasy Adventure',
 'Tornado',
 'John W. Campbell',
 'Whooping Crane',
 'Shigin',
 'Hafnium',
 'Hawaiian Goose',
 'Ankylosaurus',
 'Andriyivskyy Descent',
 'Supermarine Spitfire',
 'Carl Jung',
 'Fine art',
 'Arithmetic',
 'Taj Mahal',
 'Social capital',
 'Nepal',
 'Book of Common Prayer',
 'African Buffalo',
 'Luminiferous aether',
 'Chicago',
 'Giza pyramid complex',
 'Lyme disease',
 'Humour',
 'Imperative programming',
 'Brain',
 'Coconut',
 'Tern',
 'Wars of the Roses',
 'Davis Cup',
 'Great Tit',
 'Wave–particle duality',
 'Jean Charles de Menezes',
 'Local community',
 'Music of Spain',
 'Terik',
 'Bernard Hinault',
 "Hickman's potentilla",
 'Eris (dwarf planet)',
 'Intelligence',
 'Margaret Sanger',
 'Bretton Woods system',
 'Glastonbury Festival',
 'Oxygen',
 'Gerald Ford',
 'Antarctic Treaty System',
 'John Locke',
 'Martin Luther King, Jr.',
 'Six-party talks',
 'Coot',
 'Nint

In [4]:

len(article_titles)

4604

In [5]:
# Make the request
r = requests.get('https://rationalwiki.org/wiki/Main_Page') # /ip: Returns the requester's IP Address.

print('Response status code: {0}\n'.format(r.status_code))
print('Response headers: {0}\n'.format(r.headers))
print('Response body: {0}'.format(r.text))

Response status code: 200

Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Mon, 13 Nov 2023 21:46:04 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '11237', 'Connection': 'keep-alive', 'X-Content-Type-Options': 'nosniff', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie', 'Expires': 'Mon, 13 Nov 2023 21:46:04 GMT', 'X-Request-Id': 'ffc8d7ee6aab410119b9adff', 'Last-Modified': 'Thu, 02 Nov 2023 17:49:17 GMT', 'Content-Encoding': 'gzip', 'X-Varnish': '162442082', 'Age': '0', 'Via': '1.1 varnish (Varnish/6.2)', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate', 'Accept-Ranges': 'bytes'}

Response body: 
<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>RationalWiki</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","F

In [6]:

def construct_url_RationalWiki(title):
    if not isinstance(title, str):
        return None  # Return None or handle the error as appropriate

    normalized_title = title.lower().replace(' ', '_')  # Convert title to lowercase and replace spaces with underscores
    return f"https://rationalwiki.org/wiki/{normalized_title}"

def construct_url_Infogalactic(title):
    if not isinstance(title, str):
        return None  # Return None or handle the error as appropriate
    normalized_title = title.lower().replace(' ', '_')  # Convert title to lowercase and replace spaces with underscores
    return f"https://infogalactic.com/info/{normalized_title}"

def construct_url_Conservapedia(title):
    normalized_title = title.lower().replace(' ', '_')  # Convert title to lowercase and replace spaces with underscores
    return f"https://www.conservapedia.com/{normalized_title}"


In [7]:
#WARNING - This cell may take a long time to run (90min) - risk of blocked IP address because of too many requests
results = []

for title in article_titles:
    url = construct_url_RationalWiki(title)
    response = requests.get(url)

    # Store the title and the response status code
    results.append({'Article Title': title, 'Response Status': response.status_code})

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Optionally, save the DataFrame to a CSV file
df.to_csv('article_status_results.csv', index=False)

print(df)

                    Article Title  Response Status
0                  Second Crusade              404
1                  Navassa Island              404
2                Evan Rachel Wood              404
3     Tropical Storm Henri (2003)              404
4         Final Fantasy Adventure              404
...                           ...              ...
4599                      Réunion              404
4600                       Flower              404
4601                     Banknote              404
4602               Weyto language              404
4603                    Marseille              404

[4604 rows x 2 columns]


In [6]:

df['Response Status'].value_counts()


NameError: name 'df' is not defined

In [38]:
#WARNING - This cell may take a long time to run (12hours) - risk of blocked IP address because of too many requests
results3 = []

remaining_titles = article_titles[165:]
for index, title in enumerate(remaining_titles, start=165):
    while True:
        try : 
            url = construct_url_Conservapedia(title)
            response = requests.get(url)
            break
        except requests.ConnectionError:
            print(f"Connection error for URL: {url}. Retrying in 10 seconds...")
            time.sleep(10)  # Wait for 30 seconds before retrying

    # Store the title and the response status code
    results3.append({'Article Title': title, 'Response Status': response.status_code})
    
    # Scrape according to policy
    time.sleep(5)

    # Save every 5 articles
    if (index + 1) % 5 == 0:
        temp_df = pd.DataFrame(results3)
        temp_df.to_csv(f'article_status_partial_{index + 1}.csv', index=False)
        print(f'Saved progress at article {index + 1}')

# Create a DataFrame from the results
df3 = pd.DataFrame(results3)

# Save the final DataFrame to a CSV file
df3.to_csv('article_status_results_conservapedia.csv', index=False) 

print(df3)

Saved progress at article 170
Saved progress at article 175
Saved progress at article 180
Saved progress at article 185
Saved progress at article 190
Connection error for URL: https://www.conservapedia.com/cerebellum. Retrying in 30 seconds...
Saved progress at article 195
Saved progress at article 200
Connection error for URL: https://www.conservapedia.com/space_exploration. Retrying in 30 seconds...
Saved progress at article 205
Saved progress at article 210
Saved progress at article 215
Connection error for URL: https://www.conservapedia.com/gordon_brown. Retrying in 30 seconds...
Connection error for URL: https://www.conservapedia.com/lisbon. Retrying in 30 seconds...
Saved progress at article 220
Connection error for URL: https://www.conservapedia.com/chess. Retrying in 30 seconds...
Saved progress at article 225
Connection error for URL: https://www.conservapedia.com/otto_von_bismarck. Retrying in 30 seconds...
Saved progress at article 230
Saved progress at article 235
Saved pro

In [4]:
last_articles = pd.read_csv('article_status_partial_565.csv')
article_status_partial_165 = pd.read_csv('article_status_partial_165.csv')
article_status_partial_565 = pd.concat([
    article_status_partial_165,
    last_articles
], ignore_index=True)

article_status_partial_565.to_csv('article_status_partial_565.csv', index=False)



In [7]:

def get_article_index(title):
    return article_titles.index(title)
print(get_article_index('Final Fantasy Adventure'))

4


In [8]:
'''for title in article_titles:
    url = construct_url(title)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')    
    paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

    for para in paragraphs:
        for edit_link in para.find_all('span', class_="mw-editsection"): 
            edit_link.decompose()
        # Extract and print the text
        if para.get_text(strip=True):
         print(para.get_text())

    # Extract and process the text as needed
    # ...'''
    

print(article_titles[1681])
url = construct_url_RationalWiki(article_titles[1681])
response = requests.get(url)
#print('Response status code: {0}\n'.format(response.status_code))
#print('Response headers: {0}\n'.format(response.headers))
#print('Response body: {0}'.format(response.text))
soup = BeautifulSoup(response.content, 'html.parser')    
paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

for para in paragraphs:
    for edit_link in para.find_all('span', class_="mw-editsection"): 
        edit_link.decompose()
    # Extract and print the text
    if para.get_text(strip=True):
        print(para.get_text())      

Jamaica
Jamaica
Jamaica is a small island nation in the Caribbean. Along with 15 other countries, its head of state lives in a large palace in England. Jamaica is the birthplace of the Rastafari movement, a religion which posits that Haile Selassie of Ethiopia is God (or "Jah") and that enlightenment is achieved through smoking lots and lots of pot. It's also known for its really really fast sprinters, fast bowlers, excellent musicians, and bobsled team.

Homophobia
Unfortunately, it is also ranked consistently as one of the most homophobic countries in the world.[1] Male homosexual activity is illegal with a penalty of up to ten years in prison, and to make matters worse, the country has a serious problem with violent attacks on gay and transgender people. Examples include the murders of Oshane Gordon, a 16-year-old who was stabbed to death in his home in 2011, and Dwayne Jones, a 16-year-old trans woman who was killed in 2013, reportedly during a party in which somebody recognised he

In [9]:
print(article_titles[1681])
url = construct_url_Infogalactic(article_titles[1681])
response = requests.get(url)
#print('Response status code: {0}\n'.format(response.status_code))
#print('Response headers: {0}\n'.format(response.headers))
#print('Response body: {0}'.format(response.text))
soup = BeautifulSoup(response.content, 'html.parser')    
paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

for para in paragraphs:
    for edit_link in para.find_all('span', class_="mw-editsection"): 
        edit_link.decompose()
    # Extract and print the text
    if para.get_text(strip=True):
        print(para.get_text()) 

Jamaica
403 Forbidden


In [10]:
print(article_titles[1681])
url = construct_url_Conservapedia(article_titles[1681])
response = requests.get(url)
#print('Response status code: {0}\n'.format(response.status_code))
#print('Response headers: {0}\n'.format(response.headers))
#print('Response body: {0}'.format(response.text))


Jamaica


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
#Create the dataframe for rationalwiki.
df = pd.read_csv('article_status_results_RationalWiki.csv')
df_rationalwiki_articles = df[df['Response Status'] == 200]
df_rationalwiki_articles = df_rationalwiki_articles.drop(columns=['Response Status'])
df_rationalwiki_articles = df_rationalwiki_articles.rename(columns={'Article Title': 'title'})
display(df_rationalwiki_articles)

,title
19,Nepal
22,Luminiferous aether
23,Chicago
25,Lyme disease
26,Humour
...,...
4574,Bahrain
4578,Flood
4586,Neptune
4588,Hinduism


In [12]:
#Functions to extract the plain text of the website https://rationalwiki.org/ for every page that are present
# in both RationalWiki and Wikispedia dataset

def extract_text_rationalwiki(url):
    response = requests.get(url)
    if response.status_code != 200:
        return "Error: Unable to fetch article"

    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove the specific navigation box
    nav_box = soup.find('div', {'role': 'navigation', 'aria-labelledby': 'Nations_of_the_world-navigationbox'})
    if nav_box:
        nav_box.decompose()

    paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    
    text_content = ''
    for para in paragraphs:
        # Remove 'edit' links
        for edit_link in para.find_all('span', class_="mw-editsection"):
            edit_link.decompose()

        # Extract text
        text = para.get_text(strip=True)
        if text:
            text_content += text + '\n'
    
    return text_content

    
# Main function to update DataFrame with text content
def update_df_with_text_content(df, csv_filename):
    texts = []
    for title in df['title']:
        url = construct_url_RationalWiki(title)
        if url is not None:
            text = extract_text_rationalwiki(url)
        else:
            text = "Invalid URL or title"  # Handle the case where URL is None
        texts.append(text)
        time.sleep(1)  # Respectful crawl delay
    df['text_content'] = texts

    # Save the updated DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    return df


In [13]:
#WARNING - This cell takes a long time to run
# Create the dataframe for rationalwiki with the plain text of articles.
df_rationalwiki_articles = update_df_with_text_content(df_rationalwiki_articles,'rationalwiki_articles.csv' )
print(df_rationalwiki_articles) 

KeyboardInterrupt: 

In [81]:
#Get articles titles on Infogalactic that are in our Wikispedia Dataset 
 ##WARNING - This cell may take a long time to run (90min) - risk of blocked IP address because of too many requests
results = []
for title in article_titles:
    url = construct_url_Infogalactic(title)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers, timeout = 10)
    # Store the title and the response status code
    results.append({'Article Title': title, 'Response Status': response.status_code})

# Create a DataFrame from the results
df_infogalactic_titles = pd.DataFrame(results)

# Optionally, save the DataFrame to a CSV file
df_infogalactic_titles.to_csv('infogalactic_article_status_results.csv', index=False)

print(df_infogalactic_titles)

#How many corresponding titles beetween InfoGalactic and wikispedia
df_infogalactic_titles['Response Status'].value_counts()

                    Article Title  Response Status
0                  Second Crusade              200
1                  Navassa Island              404
2                Evan Rachel Wood              404
3     Tropical Storm Henri (2003)              404
4         Final Fantasy Adventure              404
...                           ...              ...
4599                      Réunion              200
4600                       Flower              200
4601                     Banknote              200
4602               Weyto language              200
4603                    Marseille              200

[4604 rows x 2 columns]


In [11]:

#Create the dataframe for InfoGalactic.
df_infogalactic_titles = pd.read_csv('infogalactic_article_status_results.csv')
df_infogalactic_titles = df_infogalactic_titles[df_infogalactic_titles['Response Status'] == 200]
df_infogalactic_titles = df_infogalactic_titles.drop(columns=['Response Status'])
df_infogalactic_titles = df_infogalactic_titles.rename(columns={'Article Title': 'title'})
display(df_infogalactic_titles)

,title
0,Second Crusade
5,Tornado
7,Whooping Crane
8,Shigin
9,Hafnium
...,...
4599,Réunion
4600,Flower
4601,Banknote
4602,Weyto language


In [12]:
# Function to get a random RationalWiki article title
def get_random_rationalwiki_title():
    response = requests.get('https://rationalwiki.org/wiki/Special:Random')
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.title.string.split(' - RationalWiki')[0]

# Function to check if a title exists on InfoGalactic
def check_infogalactic(title):
    url = construct_url_Infogalactic(title)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers, timeout = 10)
    return title in response.text


In [16]:
# Initialize an empty DataFrame
training_articles = pd.DataFrame(columns=['Title'])

# Loop until collection of 3000 unique articles
while len(training_articles) < 3000:
    random_title = get_random_rationalwiki_title()

    # Check conditions
    if random_title not in article_titles and random_title not in training_articles['Title'].values and check_infogalactic(random_title):
        # Add to DataFrame if conditions are met
        training_articles.loc[len(training_articles)] = [random_title]

        # Save to CSV file after every 100 additions
    if len(training_articles) % 100 == 0:
        training_articles.to_csv('training_articles_title.csv', index=False)

print("Collection complete. Final list saved to 'articles_list.csv'.")


KeyboardInterrupt: 